# Data Wrangling: Join, Combine, and Reshape

First, I introduce the concept of hierarchical indexing in pandas, which is used extensively in some of these operations. I then dig into the particular data manipulations. You can see various applied usages of these tools in [Ch 13: Data Analysis Examples](https://wesmckinney.com/book/data-wrangling.html#data-analysis-examples).


# [Hierarchical Indexing](https://wesmckinney.com/book/data-wrangling.html#pandas_hierarchical)

_Hierarchical indexing_ is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Another way of thinking about it is that it provides a way for you to work with higher dimensional data in a lower dimensional form. Let’s start with a simple example: create a Series with a list of lists (or arrays) as the index:

In [28]:
import pandas as pd
import numpy as np

In [29]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.553010
   2    0.288661
   3    0.012974
b  1    0.875347
   3    0.442858
c  1    0.156226
   2    0.886104
d  2    0.688527
   3    0.583261
dtype: float64

What you’re seeing is a prettified view of a Series with a `MultiIndex` as its index. The “gaps” in the index display mean “use the label directly above”:



In [30]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called _partial_ indexing is possible, enabling you to concisely select subsets of the data:

In [31]:
data["b"]

1    0.875347
3    0.442858
dtype: float64

In [32]:
data["b":"c"]

b  1    0.875347
   3    0.442858
c  1    0.156226
   2    0.886104
dtype: float64

In [33]:
data.loc[["b","d"]]

b  1    0.875347
   3    0.442858
d  2    0.688527
   3    0.583261
dtype: float64

Selection is even possible from an “inner” level. Here I select all of the values having the value `2` from the second index level:



In [34]:
data.loc[:, 2]

a    0.288661
c    0.886104
d    0.688527
dtype: float64

Hierarchical indexing plays an important role in reshaping data and in group-based operations like forming a pivot table. For example, you can rearrange this data into a DataFrame using its `unstack` method:



In [35]:
data.unstack()

,1,2,3
a,0.553010,0.288661,0.012974
b,0.875347,NaN,0.442858
c,0.156226,0.886104,NaN
d,NaN,0.688527,0.583261


The inverse operation of `unstack` is `stack`:

In [36]:
data.unstack().stack()

a  1    0.553010
   2    0.288661
   3    0.012974
b  1    0.875347
   3    0.442858
c  1    0.156226
   2    0.886104
d  2    0.688527
   3    0.583261
dtype: float64

`stack` and `unstack` will be explored in more detail later in [Reshaping and Pivoting](https://wesmckinney.com/book/data-wrangling.html#prep_reshape).


With a DataFrame, either axis can have a hierarchical index:



In [37]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [38]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

These names supersede the `name` attribute, which is used only with single-level indexes.


 - Be careful to note that the index names `"state"` and `"color"` are not part of the row labels (the `frame.index` values).
 

You can see how many levels an index has by accessing its `nlevels` attribute:



In [39]:
frame.index.nlevels

2

With partial column indexing you can similarly select groups of columns:

In [40]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A `MultiIndex` can be created by itself and then reused; the columns in the preceding DataFrame with level names could also be created like this:

In [41]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting Levels
At times you may need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The `swaplevel` method takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):


In [42]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` by default sorts the data lexicographically using all the index levels, but you can choose to use only a single level or a subset of levels to sort by passing the `level` argument. For example:



In [43]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [44]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

 - Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level—that is, the result of calling `sort_index(level=0)` or `sort_index()`.



## Summary Statistics by Level
Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns, like so:



In [45]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [46]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

We will discuss groupby in much more detail later in [Ch 10: Data Aggregation and Group Operations](https://wesmckinney.com/book/data-aggregation.html).

## Indexing with a DataFrame's columns

It’s not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame’s columns. Here’s an example DataFrame:


In [47]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s `set_index` function will create a new DataFrame using one or more of its columns as the index:



In [48]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default, the columns are removed from the DataFrame, though you can leave them in by passing `drop=False` to `set_index`:



In [49]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, on the other hand, does the opposite of `set_index`; the hierarchical index levels are moved into the columns:



In [50]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# Combining and Merging Datasets

Data contained in pandas objects can be combined in a number of ways:


`pandas.merge`
 - Connect rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.


`pandas.concat`
 - Concatenate or "stack" objects together along an axis.


`combine_first`
 - Splice together overlapping data to fill in missing values in one object with values from another.


I will address each of these and give a number of examples. They’ll be utilized in examples throughout the rest of the book.

## Database-Style DataFrame Joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are particularly important in relational databases (e.g., SQL-based). The `pandas.merge` function in pandas is the main entry point for using these algorithms on your data.

Let’s start with a simple example:


In [51]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

In [52]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [53]:
df2

,key,data2
0,a,0
1,b,1
2,d,2
